## Rank Score

https://leetcode.com/problems/rank-scores/solution/


In [21]:
import sqlite3

# Connect to the SQLite database (create it if it doesn't exist)
conn = sqlite3.connect('scoresX.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Create the Scores table and insert data
cursor.execute('''CREATE TABLE IF NOT EXISTS Scores (
                    id INT PRIMARY KEY,
                    score DECIMAL(5, 2)
                )''')

# Insert data into the Scores table
score_data = [
    (1, 3.50),
    (2, 3.65),
    (3, 4.00),
    (4, 3.85),
    (5, 4.00),
    (6, 3.65),
]

cursor.executemany("INSERT INTO Scores (id, score) VALUES (?, ?)", score_data)




In [27]:
## Setup if the above doesn't work

import sqlite3

# Connect to the SQLite database (create it if it doesn't exist)
conn = sqlite3.connect('scores.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()
import pandas as pd

# Create the Scores DataFrame
data = {'id': [1, 2, 3, 4, 5, 6],
        'score': [3.50, 3.65, 4.00, 3.85, 4.00, 3.65]}

scores_df = pd.DataFrame(data)

## Problem Description

Table: Scores

| Column Name | Type    |
|-------------|---------|
| id          | int     |
| score       | decimal |

`id` is the primary key (column with unique values) for this table.

Each row of this table contains the score of a game. Score is a floating point value with two decimal places.

Write a SQL query to find the rank of the scores. The ranking should be calculated according to the following rules:

- The scores should be ranked from the highest to the lowest.
- If there is a tie between two scores, both should have the same ranking.
- After a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no holes between ranks.

Return the result table ordered by score in descending order.

The result format is in the following example:

| score | rank |
|-------|------|

### Example

Example 1:

Input:
Scores table:
| id | score |
|----|-------|
| 1  | 3.50  |
| 2  | 3.65  |
| 3  | 4.00  |
| 4  | 3.85  |
| 5  | 4.00  |
| 6  | 3.65  |

Output:
| score | rank |
|-------|------|
| 4.00  | 1    |
| 4.00  | 1    |
| 3.85  | 2    |
| 3.65  | 3    |
| 3.65  | 3    |
| 3.50  | 4    |

Explanation:
In Example 1, the scores are ranked from highest to lowest. There is a tie between scores 4.00 and 4.00, so they both have rank 1. The next highest score is 3.85, which has rank 2, and so on.


In [30]:
## TODO 1: Calculate the rank of each score using Pandas

# Display the result table
result = scores_df[['score', 'rank']]
print(result)

   score  rank
0   3.50     4
1   3.65     3
2   4.00     1
3   3.85     2
4   4.00     1
5   3.65     3



### Sort the DataFrame by score in descending order
scores_df = scores_df.sort_values(by='score', ascending=False)

### Calculate the rank using rank() method with method='dense' to handle ties
scores_df['rank'] = scores_df['score'].rank(method='dense', ascending=False).astype(int)

```bash
   score  rank
2   4.00     1
4   4.00     1
3   3.85     2
1   3.65     3
5   3.65     3
0   3.50     4
```

In [22]:

# TODO Calculate the rank of scores BY Counting the number of scores that are greater than or equal to the current score


result = cursor.fetchall()

# Print the result
for row in result:
    print(row)


(4, 1)
(4, 1)
(3.85, 2)
(3.65, 3)
(3.65, 3)
(3.5, 4)


cursor.execute('''
    SELECT score, 
           (SELECT COUNT(DISTINCT score) FROM Scores s2 WHERE s2.score >= s1.score) AS rank
    FROM Scores s1
    ORDER BY score DESC
''')


In [23]:
## TODO Use Desnse Rank to calculate the rank of scores



# Fetch and display the result
result = cursor.fetchall()
for row in result:
    print(row)

(4, 1)
(4, 1)
(3.85, 2)
(3.65, 3)
(3.65, 3)
(3.5, 4)


cursor.execute("""
SELECT
  S.score,
  DENSE_RANK() OVER (
    ORDER BY
      S.score DESC
  ) AS 'rank'
FROM
  Scores S;
               """)

In [25]:
## TODO Use Inner JOIN with Count Distinct to calculate the rank of scores


# Fetch and display the result
result = cursor.fetchall()
for row in result:
    print(row)

(4, 1)
(4, 1)
(3.85, 2)
(3.65, 3)
(3.65, 3)
(3.5, 4)


Approach 3: INNER JOIN with COUNT(DISTINCT ...)
Intuition
The intuition for this approach is fundamentally the same as that for Approach 2, but the manner of implementation is completely different.

Algorithm
Join the Scores table to itself in such a manner that for each score we get all rows having a score greater than or equal to this score.
Group query rows by id and score value.
Count the number of distinct scores greater than or equal to the score used in the join condition (this is the ranking).
Order the result set by the score value.
Implementation

The intuition for this approach is fundamentally the same as that for Approach 2, but the manner of implementation is completely different.

Algorithm
Join the Scores table to itself in such a manner that for each score we get all rows having a score greater than or equal to this score.
Group query rows by id and score value.
Count the number of distinct scores greater than or equal to the score used in the join condition (this is the ranking).
Order the result set by the score value.


```sql
SELECT
  S.score,
  COUNT(DISTINCT T.score) AS 'rank'
FROM
  Scores S
  INNER JOIN Scores T ON S.score <= T.score
GROUP BY
  S.id,
  S.score
ORDER BY
  S.score DESC;
```


cursor.execute("""
    SELECT s1.score, COUNT(DISTINCT s2.score) AS rank
               FROM Scores s1
                INNER JOIN Scores s2 ON s1.score <= s2.score
                GROUP BY s1.id, s1.score
                ORDER BY s1.score DESC;
               
               
               """)

In [12]:

# Close the cursor and the database connection
cursor.close()
conn.close()